This doesnt run on the server but was used in the Bachelor Thesis to plot and extract current measurements.
It's there for documentation purposes.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

columns = ["Timestamp(ms)", "Current(uA)", "D0", "D1"] # Extract only those columns
base_path = r"C:\Users\jzurn\Documents\Studium\HBRS\Bachelorarbeit\Messungen"
filename = "v1_2_ECDHE_PSK_WITH_AES_128_GCM_SHA256_noCID_x25519_PSK"
file_path = f"{base_path}\ppk_csv\{filename}.csv"
dtls_version = "DTLS 1.2"

df = pd.read_csv(file_path, usecols=columns)

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)       
pd.set_option("display.max_colwidth", None)  

df["Current(uA)"] = df["Current(uA)"] / 1000  # Convert µA to mA
df = df.rename(columns={"Current(uA)": "Current(mA)"}) # Rename column
df["D0"] = pd.to_numeric(df["D0"], errors="coerce").fillna(0).astype(int) # Converts values to numeric unless non-numeric then NaN (column contains also X as value)
df["D1"] = pd.to_numeric(df["D1"], errors="coerce").fillna(0).astype(int)  # Convert D1 to numeric

In [ ]:
# Generate group numbers for D0 and D1
df["CoAP_Interval"] = ((df["D0"] == 1) & (df["D0"].shift(fill_value=0) == 0)).cumsum() * (df["D0"] == 1)
df["Handshakes"] = ((df["D1"] == 1) & (df["D1"].shift(fill_value=0) == 0)).cumsum() * (df["D1"] == 1)

# Calculate statistics for D0
current_coap = df[df["D0"] == 1].groupby("CoAP_Interval").agg(
    Start_Time=("Timestamp(ms)", "first"),
    End_Time=("Timestamp(ms)", "last"),
    Avg_Current=("Current(mA)", "mean"),
    Peak_Current=("Current(mA)", "max")
)

# Add derived columns for D0
current_coap["Time_Diff(s)"] = (current_coap["End_Time"] - current_coap["Start_Time"]) / 1000
current_coap["Total_Charge(mC)"] = current_coap["Avg_Current"] * current_coap["Time_Diff(s)"]

# Calculate statistics for D1
current_handshakes = df[df["D1"] == 1].groupby("Handshakes").agg(
    Start_Time=("Timestamp(ms)", "first"),
    End_Time=("Timestamp(ms)", "last"),
    Avg_Current=("Current(mA)", "mean"),
    Peak_Current=("Current(mA)", "max")
)

# Add derived columns for D1
current_handshakes["Time_Diff(s)"] = (current_handshakes["End_Time"] - current_handshakes["Start_Time"]) / 1000
current_handshakes["Total_Charge(mC)"] = current_handshakes["Avg_Current"] * current_handshakes["Time_Diff(s)"]

# Reset index for D1 and rename the group column
#current_handshakes = current_handshakes.reset_index().rename(columns={"GPIO_high_D1": "Group_coap"})
current_coap=current_coap.round(2)
current_handshakes=current_handshakes.round(2)

output_file = f"{base_path}\python_adjusted\combined_data\coap_cycles\{filename}_py.csv"
current_coap.to_csv(output_file, index=False)

output_file = f"{base_path}\python_adjusted\combined_data\handshakes\{filename}_py.csv"
current_handshakes.to_csv(output_file, index=False)


print(current_coap)
print("\n")
print(current_handshakes)


In [ ]:
# Plot Total Charge (mC) and Peak Current (mA) for D0 groups
plt.figure(figsize=(10, 9))  # Set figure size
plt.xlim(current_coap.index[0] - 0.5, current_coap.index[-1] + 0.5)

parts = filename.split("_")
cipher_suite_end_index = parts.index("noCID") if "noCID" in parts else parts.index("CID")
cipher_suite = "_".join(parts[2:cipher_suite_end_index])
curve = parts[-2]
xlabel_text = f"CoAP Cycles : {dtls_version} - {cipher_suite} - {curve}"

bars_coap = plt.bar(
    current_coap.index, 
    current_coap["Total_Charge(mC)"], 
    color="skyblue", 
    label="Total Charge (mC)",
    width=0.85,  # Reduce bar width 
)

plt.plot(
    current_coap.index, 
    current_coap["Peak_Current"], 
    color="red", 
    marker="o", 
    label="Peak Current (mA)"
)

for bar, time_diff in zip(bars_coap, current_coap["Time_Diff(s)"]):  # Assuming 'Duration(s)' exists
    plt.text(
        bar.get_x() + bar.get_width() / 2,  # X-coordinate (center of the bar)
        bar.get_height() / 2,               # Y-coordinate (above the bar)
        f"{time_diff:.2f}s",                 # Format duration as seconds
        ha="center",                        # Horizontal alignment
        va="bottom",                        # Vertical alignment
        fontsize=10,
        #rotation=90                         # Rotate text to vertical
    )
    
cipher_info = f"{dtls_version} - {cipher_suite} - {curve}"
top_y = current_coap["Total_Charge(mC)"].max()  # Find the highest value in the bar data
text_y = top_y + (0.1 * top_y)  # Add a margin above the highest bar

plt.text(         
    0.5, 0.95,
    cipher_info,
    fontsize=10,
    ha="center",  # Align text to the right
    va="bottom", # Align text to the bottom
    transform=plt.gcf().transFigure,  # Use figure-relative positioning
    bbox=dict(boxstyle="round,pad=0.3", edgecolor="gray", facecolor="lightyellow")
)

# Add labels, title, and legend for D0
plt.xlabel("CoAP Cycle", fontsize=12)
plt.ylabel("Values (mC / mA)", fontsize=12)
plt.title("Total Charge and Peak Current per CoAP Request-Response Cycle", fontsize=14)

duration_legend = mlines.Line2D(
    [], [], 
    marker="o", linestyle="None", color="black", label="0.00s (Duration)"
)

plt.legend(
    handles=[bars_coap, plt.Line2D([], [], color="red", marker="o", label="Peak Current (mA)"), duration_legend],
    fontsize=10,
    loc="upper right",
)

# Customize grid and ticks
plt.grid(True, which="both", linestyle="--", linewidth=0.5)
plt.xticks(current_coap.index, rotation=45, fontsize=10)
plt.tight_layout()

plot_file = f"{base_path}\python_adjusted\plots\coap_cycles\{filename}.png"
plt.savefig(plot_file)
print(f"Saved plot to {plot_file}")
# Show the plot for D0
plt.show()